# **Sesión 3: Procesamiento Avanzado de Imágenes** ⚙️🖼️

## **Librerías**

In [1]:
import numpy as np
import cv2
import os
import imageio
from typing import List
import glob

## **Apartado A: Detección de esquinas**

El objetivo de este apartado es detectar las esquinas presentes en las imágenes de la carpeta ``data/source``.

1. **Tarea A.1**. Cree una nueva capeta llamada ``partA``, dentro de la carpeta  ``data``, con el objetivo de presentar en ella los resultados de esta parte de la práctica.
2. **Tarea A.2**. Defina y ejecute los dos métodos propuestos para cargar imágenes ``imageio_load_images()`` y ``opencv_load_images()``. Observe lo que ocurre al guardar ambas imágenes usando la misma función ``cv2.imwrite()``.
3. **Tarea A.3.** Defina la función ``harris_corner_detector()``, que servirá para la posterior aplicación sobre las imágenes de trabajo. 
4. **Tarea A.4.** Aplique la función ``harris_corner_detector()`` sobre las imágenes de trabajo. Asegúrese de que las imágenes quedan guardadas como se especifica en los comentarios.
5. **Tarea A.5.** Defina la función ``shi_tomasi_corner_detection()``, que servirá para la posterior aplicación sobre las imágenes de trabajo.
6. **Tarea A.6.** Aplique la función ``shi_tomasi_corner_detection()`` sobre las imágenes de trabajo. Asegúrese de que las imágenes quedan guardadas como se especifica en los comentarios.

### **Tarea A.1**. Cree una nueva capeta llamada ``partA``, dentro de la carpeta  ``data``, con el objetivo de presentar en ella los resultados de esta parte de la práctica.

In [2]:
# TODO Create a folder to save all partA results (inside data)
folder_name = "partA"
folder_path = os.path.join(os.path.dirname(os.getcwd()), folder_name)
os.makedirs(folder_path, exist_ok=True)

### **Tarea A.2**. Defina y ejecute los dos métodos propuestos para cargar imágenes ``imageio_load_images()`` y ``opencv_load_images()``. Observe lo que ocurre al guardar ambas imágenes usando la misma función ``cv2.imwrite()``.

In [3]:
# This initial part is to highlight that cv2.imshow() and cv2.imwrite() works well with previous BGR conversion
def imageio_load_images(filenames: List) -> List:
    '''
    Load images using imageio.imread function (RGB)
    '''
    return [imageio.v2.imread(filename) for filename in filenames]

def opencv_load_images(filenames: List) -> List:
    '''
    Load images cv2.imread function (BGR)
    '''
    return [cv2.imread(filename) for filename in filenames]

# TODO Create two sorted lists with the paths of all images in the data/source folder using glob
source_paths = sorted(glob.glob("../data/source/*"))
imageio_images = imageio_load_images(source_paths)
opencv_images = opencv_load_images(source_paths)

# TODO Last element of both image lists is a blue tennis court, so try saving them in partA folder
image_1_path = os.path.join(folder_path, "imageio.png")
cv2.imwrite(image_1_path, imageio_images[-1])
image_2_path = os.path.join(folder_path, "opencv.png")
cv2.imwrite(image_2_path, opencv_images[-1])

True

### **Tarea A.3.** Defina la función ``harris_corner_detector()``, que servirá para la posterior aplicación sobre las imágenes de trabajo. 

In [4]:
def harris_corner_detector(image: np.array, blockSize: int, ksize: int, k: float):
    '''
    Detects corners in an image using the Harris Corner Detection method.
    
    Parameters:
    - image: Input image in RGB or grayscale format
    - blockSize: Size of the neighborhood considered for corner detection
    - ksize: Aperture parameter for the Sobel operator
    - k: Harris detector free parameter in the equation

    Returns:
    - Image with detected corners highlighted in red
    '''
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Convert to float32 type
    gray = np.float32(gray)
    
    # Apply Harris corner detection
    harris = cv2.cornerHarris(gray, blockSize=blockSize, ksize=ksize, k=k)
    
    # Dilate result to mark the corners
    harris = cv2.dilate(harris, None)
    
    # Thresholding to highlight corners in red
    threshold = 0.01 * harris.max()
    image[harris > threshold] = [0, 0, 255]  # Set corners to red
    
    return image


### **Tarea A.4.** Aplique la función ``harris_corner_detector()`` sobre las imágenes de trabajo. Asegúrese de que las imágenes quedan guardadas como se especifica en los comentarios.

In [6]:
# This section is designed to to change corner detection parameters for each image
# We want to save processed image in path: Lab3/data/partA/Harris_{save_name}.jpg

blockSize = 2  # Size of neighborhood for corner detection
ksize = 3      # Aperture parameter for the Sobel operator
k = 0.05       # Harris detector free parameter

def apply_and_save_harris(save_name: str, index: int):
    # Copy first original image (replace 'path_to_first_image' with actual path)
    image = cv2.imread(source_paths[index])

    # Apply Harris Corner Detection
    harris_image = harris_corner_detector(image, blockSize=blockSize, ksize=ksize, k=k)

    # Save final image in partA folder
    cv2.imwrite(os.path.join(folder_path, f"harris_{save_name}.png"), harris_image)

apply_and_save_harris("geometry", 0) # Geometry
apply_and_save_harris("football", 1)  # Texture

### **Tarea A.5.** Defina la función ``shi_tomasi_corner_detection()``, que servirá para la posterior aplicación sobre las imágenes de trabajo.

In [7]:
# TODO Define Shi-Tomasi corner detection function
def shi_tomasi_corner_detection(image: np.array, maxCorners: int, qualityLevel:float, minDistance: int, corner_color: tuple, radius: int):
    '''
    image - Input image
    maxCorners - Maximum number of corners to return. 
                 If there are more corners than are found, the strongest of them is returned. 
                 maxCorners <= 0 implies that no limit on the maximum is set and all detected corners are returned
    qualityLevel - Parameter characterizing the minimal accepted quality of image corners. 
                   The parameter value is multiplied by the best corner quality measure, which is the minimal eigenvalue or the Harris function response. 
                   The corners with the quality measure less than the product are rejected. 
                   For example, if the best corner has the quality measure = 1500, and the qualityLevel=0.01 , then all the corners with the quality measure less than 15 are rejected
    minDistance - Minimum possible Euclidean distance between the returned corners
    corner_color - Desired color to highlight corners in the original image
    radius - Desired radius (pixels) of the circle
    '''
    # TODO Input image to Tomasi corner detector should be grayscale 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Shi-Tomasi corner detection
    corners = cv2.goodFeaturesToTrack(
        gray, 
        maxCorners=maxCorners, 
        qualityLevel=qualityLevel, 
        minDistance=minDistance
    )
    
    # Ensure corner coordinates are integers
    corners = np.int0(corners)
    
    # Draw circles around each detected corner
    for corner in corners:
        x, y = corner.ravel()  # Flatten the coordinates
        cv2.circle(image, (x, y), radius, corner_color, -1)  # Draw filled circle
    
    return image

### **Tarea A.6.** Aplique la función ``shi_tomasi_corner_detection()`` sobre las imágenes de trabajo. Asegúrese de que las imágenes quedan guardadas como se especifica en los comentarios.

In [11]:
# This section is designed to to change corner detection parameters for each image
# We want to save processed image in path: Lab3/data/partA/Shi-Tomasi_{save_name}.jpg

# First image - Purple corners and radius = 4
purple_color = (255, 0, 255)  # BGR format for purple

# Parameters for Shi-Tomasi corner detection
maxCorners = 60          # Maximum number of corners to detect (adjust as needed)
qualityLevel = 0.05      # Minimum quality of corners (between 0 and 1)
minDistance = 7         # Minimum distance between corners

def apply_and_save_shi(save_name: str, index: int):
    # Copy first original image (replace 'path_to_first_image' with actual path)
    image = cv2.imread(source_paths[index])

    # Apply Harris Corner Detection
    tomasi_image = shi_tomasi_corner_detection(image, maxCorners=maxCorners, qualityLevel=qualityLevel, minDistance=minDistance, corner_color=purple_color, radius=5)

    # Save final image in partA folder
    cv2.imwrite(os.path.join(folder_path, f"shi_{save_name}.png"), tomasi_image)

apply_and_save_shi("geometry", 0) # Geometry
apply_and_save_shi("football", 1)  # Football

C:\Users\javie\AppData\Local\Temp\ipykernel_12456\7618231.py:28: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  corners = np.int0(corners)


### **Pregunta A.1:** Realice la detección de esquinas en las otras dos imágenes de la carpeta ``data/source`` (cuyos nombres de guardado han de ser "sudoku" y "tennis") con el método de Harris

In [12]:
# Code by yourself
apply_and_save_harris("sudoku", 2)  # Sudoku
apply_and_save_harris("tennis", 3)  # Tennis court

### **Pregunta A.2:** Realice la detección de esquinas en las otras dos imágenes de la carpeta ``data/source`` (cuyos nombres de guardado han de ser "sudoku" y "tennis") con el método de Shi-Tomasi

In [13]:
# Code by yourself
apply_and_save_shi("sudoku", 2)  # Sudoku
apply_and_save_shi("tennis", 3)  # Tennis court

C:\Users\javie\AppData\Local\Temp\ipykernel_12456\7618231.py:28: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  corners = np.int0(corners)
